In [ ]:
import pandas as pd
import numpy as np
import random
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.environ import SolverFactory
import yfinance as yf
import matplotlib.pyplot as plt


In [8]:
df_ingredientes = pd.read_excel('base_dados_exercicio_1.xlsx', sheet_name='ingredientes')
df_racao = pd.read_excel('base_dados_exercicio_1.xlsx', sheet_name='racao')
df_conteudo_nutricional = pd.read_excel('base_dados_exercicio_1.xlsx', sheet_name='conteudo_nutricional')

In [ ]:
df_ingredientes = df_ingredientes.set_index('Ingrediente')
df_racao = df_racao.set_index('Ração')
df_conteudo_nutricional=df_conteudo_nutricional.set_index('Ingrediente')

In [19]:
df_ingredientes

,Disponibilidade (kg),Custo Unitário (R$/kg)
Ingrediente,,
Milho,1000,0.5
Soja,800,0.8
Suplemento Vitamínico,200,1.5


## INICIAR MODELO

In [38]:
model = pyo.ConcreteModel()

# RANGE SET
# model.A = pyo.RangeSet(0,len(retornos_ativo_treino.columns)-1)
# model.T = pyo.RangeSet(0, len(retornos_ativo_treino)-1)
model.D = pyo.RangeSet(0,1)
# PARAMETROS----------------------
# model.ATIVOS = pyo.Param(model.T,model.A, initialize=dicio_ativos)


# VARIAVEIS----------------------
model.milho = pyo.Var(range(2), bounds=(0,None))
model.soja = pyo.Var(range(2), bounds=(0,None))
model.suplemento = pyo.Var(range(2), bounds=(0,None))

## Restrições

def restricao_nutricional_racao_A(model):
    proteina = sum(model.milho[0]*5 + model.soja[0]*10 + model.suplemento[0]*2 )
    energia = sum(model.milho[0]*8 + model.soja[0]*5 + model.suplemento[0]*3 )
    
    return proteina + energia 
model.restricao_nutricional_racao_A = pyo.Constraint(model.milho, model.soja, model.suplemento,rule=restricao_nutricional_racao_A)



TypeError: Cannot apply a Set operator to an indexed Var component (milho)

In [34]:
model.pprint()

1 RangeSet Declarations
    D : Dimen=1, Size=2, Bounds=(0, 1)
        Key  : Finite : Members
        None :   True :   [0:1]

3 Var Declarations
    milho : Size=2, Index={0, 1}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
    soja : Size=2, Index={0, 1}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
    suplemento : Size=2, Index={0, 1}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals

1 Constraint Declarations
    restricao_nutricional_racao_A : Size=0, Index=D, Active=True
        Key : Lower : Body : Upper : Active

5 Declarations: D milho soja suplemento restricao_nutricional_ra

In [8]:
# RESTRICOES ----------------------
    ## - Ela diz que a proteção deve ser maior ou igual ao impacto do desvio padrão no peso do ativo
def regra_protecao(model,t,a):
    
    return model.z[t] + model.p[t,a] >= model.DELTA[a] * model.x[a]

model.con_protecao = pyo.Constraint(model.T,model.A, rule=regra_protecao)

    ## OUTRAS RESTRIÇÔES

    # 1- Cardinalidade (total ativos na carteira)
def cardinalidade(model):

    return sum(
        model.y[a] for a in model.A
    ) <= model.QUANTIDADE_ATIVOS
model.cardinalidade = pyo.Constraint(rule=cardinalidade)

    # 2 - Soma peso da carteira
def soma_peso_1(model):
    return sum(model.x[a] for a in model.A) == 1
model.soma_peso_1 = pyo.Constraint(rule=soma_peso_1)

    # 3 - x só ativa se y=1
def restr_vinculo_x_y(model, a):

    return model.x[a] <= model.y[a]

model.restr_vinculo_x_y = pyo.Constraint(model.A, rule=restr_vinculo_x_y)

    # Retrição de Peso mínimo, se y=1, x deve ser pelo menos 1%
def peso_min(model, a):
    return model.x[a] >= 0.01 * model.y[a]
model.peso_min = pyo.Constraint(model.A, rule=peso_min)




In [10]:
opt = SolverFactory('cplex', executable='C:\\CPLEX_Studio2211\\cplex\\bin\\x64_win64\\cplex.exe')
opt.options['TimeLimit'] = 480 
# resultado = opt.solve(model)
# opt.options['optimalitytarget'] = 3 ## PARA CASO DE MODELOS NAO LINEARES
opt.options['mip tolerances absmipgap'] = 0.01 
opt.options['mip tolerances mipgap'] = 0.01 
opt.options['mipgap'] = 0.01 

results = opt.solve(model, tee=True, warmstart=True)
print("-------FIM----------")



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\DECIV\AppData\Local\Temp\tmpdk6fxhsf.cplex.log' open.
CPLEX> New value for mixed integer optimality gap tolerance: 0.01
CPLEX> New value for time limit in seconds: 480
CPLEX> New value for absolute mixed integer optimality gap tolerance: 0.01
CPLEX> New value for mixed integer optimality gap tolerance: 0.01
CPLEX> Problem 'C:\Users\DECIV\AppData\Local\Temp\tmpcddflutw.pyomo.lp' read.
Read time = 0.02 sec. (1.51 ticks)
CPLEX> MIP start file 'C:\Users\DECIV\AppData\Local\Temp\tmp3_v0d3hi.cplex.mst' read.
CPLEX> Problem name         : C:\Users\DECIV\AppData\Local\Temp\tmpcddflutw.pyomo.l